In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
from lxml import etree

In [3]:
def Xpath(url,path):
  Dict_Headers = ({'User-Agent':
      'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 \
      (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',\
      'Accept-Language': 'en-US, en;q=0.5'})
  # Gets the requried data https browser's address bar
  webPage = requests.get(url,Dict_Headers)
  # Creating a soup Object from the html content
  Scraping = BeautifulSoup(webPage.content, "html.parser") 
  # Conveting Soup object to etree object for Xpath processing
  documentObjectModel = etree.HTML(str(Scraping)) 
  return (documentObjectModel.xpath(path)[0].text)

In [15]:
def nbListes(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    s = soup.find('div', class_='calendar-widget-container')
    s2 = s.find_all('ul')
    return len(s2)

def nbElementsDansChaqueListe(url):
    l = []
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    s = soup.find('div', class_='calendar-widget-container')
    s2 = s.find_all('ul')
    for ul in s2:
        s3 = ul.find_all('li')
        l.append(len(s3))
    return l
        
def homeTeam(url):
    l = []
    for i in range(1, nbListes(url)+1):
        for j in range(1, nbElementsDansChaqueListe(url)[i-1]+1):
            l.append(Xpath(url, f'/html/body/main/div[3]/div[2]/div/div[2]/ul[{i}]/li[{j}]/a/div[1]/div/span[1]'))
    return l

def awayTeam(url):
    l = []
    for i in range(1, nbListes(url)+1):
        for j in range(1, nbElementsDansChaqueListe(url)[i-1]+1):
            l.append(Xpath(url, f'/html/body/main/div[3]/div[2]/div/div[2]/ul[{i}]/li[{j}]/a/div[3]/div/span[1]'))
    return l

In [18]:
def homeGoal(url):
    l = []
    for i in range(1, nbListes(url)+1):
        for j in range(1, nbElementsDansChaqueListe(url)[i-1]+1):
            if Xpath(url, f'/html/body/main/div[3]/div[2]/div/div[2]/ul[{i}]/li[{j}]/a/div[2]/span/span[3]') == None:
                l.append(None)
            else:
                l.append(int(Xpath(url, f'/html/body/main/div[3]/div[2]/div/div[2]/ul[{i}]/li[{j}]/a/div[2]/span/span[3]')))
    return l

def awayGoal(url):
    l = []
    for i in range(1, nbListes(url)+1):
        for j in range(1, nbElementsDansChaqueListe(url)[i-1]+1):
            if Xpath(url, f'/html/body/main/div[3]/div[2]/div/div[2]/ul[{i}]/li[{j}]/a/div[2]/span/span[3]') == None:
                l.append(None)
            else:
                l.append(int(Xpath(url, f'/html/body/main/div[3]/div[2]/div/div[2]/ul[{i}]/li[{j}]/a/div[2]/span/span[3]')))
    return l

In [19]:
import csv

In [ ]:
header = ['Season','Matchday','HomeTeam', 'AwayTeam', 'HTG', 'ATG']
with open('all_ligue1_matches.csv', 'w', encoding='UTF8', newline='') as f:
    fwriter = csv.writer(f)
    fwriter.writerow(header)
    for annee in range(1993, 2022):
        if annee == 2019:
            # annee covid seulement 28 journees terminees
            for i in range(1,29):
                url = f'https://www.ligue1.fr/calendrier-resultats?seasonId=2019-2020&matchDay={i}'
                for j in range(0,sum(nbElementsDansChaqueListe(url))):
                    fwriter.writerow(['2019-2020', i, homeTeam(url)[j], awayTeam(url)[j], homeGoal(url)[j], awayGoal(url)[j]])
        elif annee == 1997 or annee == 1998 or annee == 1999 or annee == 2000 or annee == 2001:
            # annees ligue 1 composee de 18 equipes donc 34 journees
            for i in range(1,35):
                url = f'https://www.ligue1.fr/calendrier-resultats?seasonId={annee}-{annee+1}&matchDay={i}'
                for j in range(0,sum(nbElementsDansChaqueListe(url))):
                    fwriter.writerow([f'{annee}-{annee+1}', i, homeTeam(url)[j], awayTeam(url)[j], homeGoal(url)[j], awayGoal(url)[j]])
        elif annee == 2021:
            # annee actuel seulement 35 journees terminees le 7 mai 2022
            for i in range(1,36):
                url = f'https://www.ligue1.fr/calendrier-resultats?seasonId=2021-2022&matchDay={i}'
                for j in range(0,sum(nbElementsDansChaqueListe(url))):
                    fwriter.writerow(['2021-2022', i, homeTeam(url)[j], awayTeam(url)[j], homeGoal(url)[j], awayGoal(url)[j]])
        else:
            for i in range(1,39):
                url = f'https://www.ligue1.fr/calendrier-resultats?seasonId={annee}-{annee+1}&matchDay={i}'
                for j in range(0,sum(nbElementsDansChaqueListe(url))):
                    fwriter.writerow([f'{annee}-{annee+1}', i, homeTeam(url)[j], awayTeam(url)[j], homeGoal(url)[j], awayGoal(url)[j]])